## install moveit_commander

```
sudo apt-get install ros-"DISTRO"-moveit-commander
pip install pyassimp
```

## reference

https://ros-planning.github.io/moveit_tutorials/doc/move_group_python_interface/move_group_python_interface_tutorial.html

## Before Running Moveit 

```
# run gazebo
roslaunch ur_gazebo ur5_robotiq.launch sim:=true limited:=true
# run moveit
roslaunch ur5_moveit_config ur5_moveit_planning_execution.launch sim:=true limited:=true
# run rviz
roslaunch ur5_moveit_config moveit_rviz.launch config:=true
```


In [1]:
from __future__ import print_function
from six.moves import input

import os
ROS_DISTRO = os.environ['ROS_DISTRO']

import sys
ros_python_path = '/opt/ros/{}/lib/python2.7/dist-packages'.format(ROS_DISTRO)
if not ros_python_path in sys.path:
    sys.path.append(ros_python_path)

import rospy
import copy

#for moveit
from moveit_commander.conversions import pose_to_list
from std_msgs.msg import String
import geometry_msgs.msg
import moveit_commander
import moveit_msgs.msg
from math import pi

# for gripper
from robotiq_2f_gripper_msgs.msg import CommandRobotiqGripperFeedback, CommandRobotiqGripperResult, CommandRobotiqGripperAction, CommandRobotiqGripperGoal
from robotiq_2f_gripper_control.robotiq_2f_gripper_driver import Robotiq2FingerGripperDriver as Robotiq
import actionlib

moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node('arm_ur5_gripper_robotiq_client', anonymous=True)

# Provides information such as the robot’s kinematic model and the robot’s current joint states
robot = moveit_commander.RobotCommander()

# provides a remote interface for getting, setting, and updating the robot’s internal understanding of the surrounding world
# 플래닝할때, 주변 장애물을 반영할 수 있게, 아래의 object를 통해 환경에 장애물을 추가할 수 있다.
scene = moveit_commander.PlanningSceneInterface()

# plan 한 trajectory를 rviz화면에 시뮬레이션해줌.
# "roslaunch ur5_moveit_config moveit_rviz.launch config:=true" 을 통해 미리 rviz를 띄어놔야 함.
display_trajectory_publisher = rospy.Publisher('/move_group/display_planned_path',
                                               moveit_msgs.msg.DisplayTrajectory,
                                               queue_size=20)

# move_group 객체를 통해 각 joint(move_element)의 desired states를 지정할 수 있고, moveit은 이 값을 바탕으로 planning을 해줌.
group_name = "manipulator"
move_group = moveit_commander.MoveGroupCommander(group_name)

# We can get the name of the reference frame for this robot:
planning_frame = move_group.get_planning_frame()
print("============ Planning frame: %s" % planning_frame)

# We can also print the name of the end-effector link for this group:
eef_link = move_group.get_end_effector_link()
print("============ End effector link: %s" % eef_link)

# We can get a list of all the groups in the robot:
group_names = robot.get_group_names()
print("============ Available Planning Groups:", robot.get_group_names())

# Sometimes for debugging it is useful to print the entire state of the
# robot:
print("============ Printing robot state")
print(robot.get_current_state())
print("")

joint_goal = move_group.get_current_joint_values()
joint_goal = [2.928491837446625,
 -1.3835698676261812,
 1.716120040745281,
 -2.0037368618694433,
 -1.5245347859526337,
 2.887431079190039]
move_group.go(joint_goal, wait=True)
move_group.stop()

action_name = rospy.get_param('~action_name', 'command_robotiq_action')
print(action_name)
robotiq_client = actionlib.SimpleActionClient(action_name, CommandRobotiqGripperAction)
robotiq_client.wait_for_server()
print("Client test: Starting sending goals")

joint_goal = move_group.get_current_joint_values()
joint_goal = [2.9412768042098367,
 -1.1789994888028392,
 1.8574884240683716,
 -2.35008618243274,
 -1.5256609547041684,
 2.900080107538237]
move_group.go(joint_goal, wait=True)
move_group.stop()

goal = CommandRobotiqGripperGoal()
goal.emergency_release = False
goal.stop = False
goal.position = 0.0
goal.speed = 0.1
goal.force = 5.0
robotiq_client.send_goal(goal)
robotiq_client.wait_for_result()

============ Planning frame: world
============ End effector link: ee_link
============ Available Planning Groups: ['endeffector', 'manipulator']
============ Printing robot state
joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  name: 
    - shoulder_pan_joint
    - shoulder_lift_joint
    - elbow_joint
    - wrist_1_joint
    - wrist_2_joint
    - wrist_3_joint
    - finger_joint
    - left_inner_finger_joint
    - left_inner_knuckle_joint
    - right_inner_knuckle_joint
    - right_outer_knuckle_joint
    - right_inner_finger_joint
  position: [0.0004621523069365807, 0.00872556623279408, 0.00019296241580590845, 0.0007623052440530742, -7.819740866477076e-05, 5.757569569020404e-05, 0.0039698816160340655, -0.0039698816160340655, 0.0039698816160340655, 0.0039698816160340655, 0.0039698816160340655, -0.0039698816160340655]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nse

True

In [2]:
goal = CommandRobotiqGripperGoal()
goal.emergency_release = False
goal.stop = False
goal.position = 0.5
goal.speed = 0.1
goal.force = 5.0
robotiq_client.send_goal(goal)
robotiq_client.wait_for_result()

True

In [3]:
goal = CommandRobotiqGripperGoal()
goal.emergency_release = False
goal.stop = False
goal.position = 0.0
goal.speed = 0.1
goal.force = 0.2
robotiq_client.send_goal(goal)
robotiq_client.wait_for_result()

True

In [5]:
joint_goal = move_group.get_current_joint_values()
joint_goal[0] -= 0.05
move_group.go(joint_goal, wait=True)
move_group.stop()

In [2]:
joint_goal = move_group.get_current_joint_values()
joint_goal = [2.928491837446625,
 -1.3835698676261812,
 1.716120040745281,
 -2.0037368618694433,
 -1.5245347859526337,
 2.887431079190039]
move_group.go(joint_goal, wait=True)
move_group.stop()

In [9]:
joint_goal = move_group.get_current_joint_values()
joint_goal = [2.9412768042098367,
 -1.1789994888028392,
 1.8574884240683716,
 -2.35008618243274,
 -1.5256609547041684,
 2.900080107538237]
move_group.go(joint_goal, wait=True)
move_group.stop()